In [28]:
import cv2
import os
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from torchvision import models, transforms




: 

In [29]:
video_path = "./vid.mp4"

vid = "./HWvideo.mp4"


In [30]:
COCO_CLASS_NAMES = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", "traffic light",
    "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow",
    "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
    "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard",
    "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
    "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote", "keyboard",
    "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase",
    "scissors", "teddy bear", "hair drier", "toothbrush"
]


In [31]:
print(torch.backends.mps.is_available())

True


In [32]:


model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to('cpu')
model.eval()

/opt/homebrew/anaconda3/envs/msda/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/anaconda3/envs/msda/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [37]:

model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to('cpu')
model.eval()

transform = transforms.Compose([
    transforms.ToTensor()
])

video_path = './vid.mp4'
cap = cv2.VideoCapture(video_path)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

out_video = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img_tensor = transform(frame).unsqueeze(0)

    with torch.no_grad():
        predictions = model(img_tensor)

    for box, label in zip(predictions[0]['boxes'], predictions[0]['labels']):
        if label == 3:
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            cv2.putText(frame, str(label), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("img", frame)



    # out_video.write(frame)

cap.release()
out_video.release()
cv2.destroyAllWindows()


: 

In [8]:
from ultralytics import YOLO
import torch

model = YOLO("yolov8m.pt")
torch.mps.empty_cache()

device = torch.device("cpu")

model.train(
    data="/Users/nikhilgudur/datasets/aiHub.v1i.yolov8/data.yaml",
    epochs=5,
    imgsz=256,
    batch=8,
    name="yolov8_model",
    device=device
)


metrics = model.val()



engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/Users/nikhilgudur/datasets/aiHub.v1i.yolov8/data.yaml, epochs=5, time=None, patience=100, batch=8, imgsz=256, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=yolov8_model12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, kera

train: Scanning /Users/nikhilgudur/datasets/aiHub.v1i.yolov8/train/labels.cache... 534 images, 0 backgrounds, 0 corrupt: 100%|██████████| 534/534 [00:00<?, ?it/s]
val: Scanning /Users/nikhilgudur/datasets/aiHub.v1i.yolov8/valid/labels.cache... 51 images, 0 backgrounds, 0 corrupt: 100%|██████████| 51/51 [00:00<?, ?it/s]


Plotting labels to /opt/homebrew/runs/detect/yolov8_model12/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 83 weight(decay=0.0), 90 weight(decay=0.0005), 89 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 256 train, 256 val
Using 0 dataloader workers
Logging results to /opt/homebrew/runs/detect/yolov8_model12
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5         0G      1.143      3.291      1.296         26        256: 100%|██████████| 67/67 [01:58<00:00,  1.77s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:07<00:00,  1.93s/it]

                   all         51         82      0.879      0.191      0.278      0.165


...


5 epochs completed in 0.174 hours.
Optimizer stripped from /opt/homebrew/runs/detect/yolov8_model12/weights/last.pt, 46.7MB
Optimizer stripped from /opt/homebrew/runs/detect/yolov8_model12/weights/best.pt, 46.7MB

Validating /opt/homebrew/runs/detect/yolov8_model12/weights/best.pt...
Ultralytics 8.3.1 🚀 Python-3.12.6 torch-2.4.0 CPU (Apple M2 Pro)
Model summary (fused): 236 layers, 23,206,306 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:07<00:00,  1.90s/it]


                   all         51         82      0.385      0.503      0.497      0.365
                Hammer          7         18      0.779      0.667      0.653      0.518
                Pliers          7          8      0.207      0.375      0.209      0.139
                  Rope         12         13       0.44      0.923       0.85      0.637
          Screw Driver          8         14          0          0     0.0343     0.0275
              Tool Box          6          7      0.475      0.778      0.809      0.562
                Wrench         12         22      0.407      0.273      0.424      0.307
Speed: 0.2ms preprocess, 145.9ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /opt/homebrew/runs/detect/yolov8_model12
Ultralytics 8.3.1 🚀 Python-3.12.6 torch-2.4.0 CPU (Apple M2 Pro)
Model summary (fused): 236 layers, 23,206,306 parameters, 0 gradients


val: Scanning /Users/nikhilgudur/datasets/aiHub.v1i.yolov8/valid/labels.cache... 51 images, 0 backgrounds, 0 corrupt: 100%|██████████| 51/51 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:03<00:00,  2.21it/s]


                   all         51         82      0.385      0.503      0.497      0.365
                Hammer          7         18      0.779      0.667      0.653      0.518
                Pliers          7          8      0.207      0.375      0.209      0.139
                  Rope         12         13       0.44      0.923       0.85      0.637
          Screw Driver          8         14          0          0     0.0343     0.0275
              Tool Box          6          7      0.475      0.778      0.809      0.562
                Wrench         12         22      0.407      0.273      0.424      0.307
Speed: 0.1ms preprocess, 58.8ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /opt/homebrew/runs/detect/yolov8_model122


In [34]:
print(f"Validation metrics: {metrics}")


Validation metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x41b025640>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,  

In [13]:

model = YOLO("/opt/homebrew/runs/detect/yolov8_model12/weights/best.pt")

results = model.predict(source="/Users/nikhilgudur/datasets/aiHub.v1i.yolov8/test/images", device=device,save=True)
print(f"Test Results: {results}")


image 1/26 /Users/nikhilgudur/datasets/aiHub.v1i.yolov8/test/images/000002_jpg.rf.3fe959f5175a87e988c5a5f69d779fcd.jpg: 256x256 2 Ropes, 83.8ms
image 2/26 /Users/nikhilgudur/datasets/aiHub.v1i.yolov8/test/images/000007_jpg.rf.4fea82fe27d57392d3bea4fee5d955f2.jpg: 256x256 2 Ropes, 77.4ms
image 3/26 /Users/nikhilgudur/datasets/aiHub.v1i.yolov8/test/images/000007_jpg.rf.60dbc245bf7d0b7c5455cf7333f99164.jpg: 256x256 1 Wrench, 79.1ms
image 4/26 /Users/nikhilgudur/datasets/aiHub.v1i.yolov8/test/images/000008_jpg.rf.1c9fac7314f7c395684d767630d8435a.jpg: 256x256 (no detections), 90.0ms
image 5/26 /Users/nikhilgudur/datasets/aiHub.v1i.yolov8/test/images/000008_jpg.rf.2ec3ae0f45aad59174c2139b1ab2c04c.jpg: 256x256 (no detections), 76.9ms
image 6/26 /Users/nikhilgudur/datasets/aiHub.v1i.yolov8/test/images/000008_jpg.rf.92c9368abc042a5793ad34a08e16ccc7.jpg: 256x256 1 Rope, 1 Tool Box, 76.3ms
image 7/26 /Users/nikhilgudur/datasets/aiHub.v1i.yolov8/test/images/000013_jpg.rf.e379ee03d9b72a5439ae32dbb

In [14]:
output_dir = '/opt/homebrew/runs/detect/predict3/'

for result in results:
    img_path = result.path
    img = cv2.imread(img_path)

    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        confidence = box.conf[0]
        class_id = int(box.cls[0])
        class_name = model.names[class_id]

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        label = f"{class_name} ({confidence:.2f})"
        cv2.putText(img, str(label), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    output_img_path = os.path.join(output_dir, f"bbox_{os.path.basename(img_path)}")
    cv2.imwrite(output_img_path, img)
    print(f"Saved: {output_img_path}")

Saved: /opt/homebrew/runs/detect/predict3/bbox_000002_jpg.rf.3fe959f5175a87e988c5a5f69d779fcd.jpg
Saved: /opt/homebrew/runs/detect/predict3/bbox_000007_jpg.rf.4fea82fe27d57392d3bea4fee5d955f2.jpg
Saved: /opt/homebrew/runs/detect/predict3/bbox_000007_jpg.rf.60dbc245bf7d0b7c5455cf7333f99164.jpg
Saved: /opt/homebrew/runs/detect/predict3/bbox_000008_jpg.rf.1c9fac7314f7c395684d767630d8435a.jpg
Saved: /opt/homebrew/runs/detect/predict3/bbox_000008_jpg.rf.2ec3ae0f45aad59174c2139b1ab2c04c.jpg
Saved: /opt/homebrew/runs/detect/predict3/bbox_000008_jpg.rf.92c9368abc042a5793ad34a08e16ccc7.jpg
Saved: /opt/homebrew/runs/detect/predict3/bbox_000013_jpg.rf.e379ee03d9b72a5439ae32dbb693de3a.jpg
Saved: /opt/homebrew/runs/detect/predict3/bbox_000016_jpg.rf.49caa35a4fa6c64f926aa19207624c03.jpg
Saved: /opt/homebrew/runs/detect/predict3/bbox_000017_jpg.rf.dac6266dcf1b636cbbf68b83d5ef9e09.jpg
Saved: /opt/homebrew/runs/detect/predict3/bbox_000023_jpg.rf.535ac4439de275270923da6cf2fe1430.jpg
Saved: /opt/homebrew

Detectron